In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import QuantLib as ql
import datetime

In [3]:
# set u quantlib
ql.Settings.instance().evaluationDate = ql.Date(1,2,2013)
calendar = ql.UnitedStates()

In [4]:
def convert_to_quantlib_datetime(df, date_columns):
    """
    Convert specified columns in a dataframe to QuantLib DateTime types.

    Parameters:
    - df: pandas DataFrame containing the data.
    - date_columns: list of column names to be converted.

    Returns:
    - DataFrame with specified columns converted to QuantLib DateTime types.
    """
    for col in date_columns:
        # Convert pandas datetime to QuantLib Date
        col2 = col + '_ql'
        df[col2] = df[col].apply(lambda x: ql.Date(x.day, x.month, x.year) if pd.notnull(x) else None)

    return df

In [5]:
data = pd.read_excel("data/USD_CAD_combined.xlsx")

In [6]:
# Specify the date columns to be converted
date_columns = ['File Date', 'Effective', 'Maturity', 'Trade Date']

data["File Date"] = pd.to_datetime(data["File Date"])
data["Trade Date"] = pd.to_datetime(data["Trade Date"])
data["Effective"] = pd.to_datetime(data["Effective"])
data["Maturity"] = pd.to_datetime(data["Maturity"])

data = convert_to_quantlib_datetime(df=data, date_columns=date_columns)


In [7]:

data["Days Between Trade and Effective"] = data.apply(lambda x: calendar.businessDaysBetween(x["Trade Date_ql"], x["Effective_ql"], includeFirst=False, includeLast=True), axis=1)

In [8]:
usd = data[data["Curr"] == "USD"].groupby(("Days Between Trade and Effective")).count()
cad = data[data["Curr"] == "CAD"].groupby(("Days Between Trade and Effective")).count()

In [13]:
# Filter by CD, Leg 1, Rate 2, Othr Pmnt, and T
filtered_data = data[(data['CD'] == "TR") & 
                     (data['Leg 1'].str.upper() == "FIXED") & 
                     (data['Rate 2'].isnull()) & 
                     (data['Othr Pmnt'].isnull()) & 
                     (data['T'].isin(["1Y", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y"]))]


# For USD currency, Leg 2 should be "USD-LIBOR-BBA"
# For CAD currency, Leg 2 should be "CAD-BA-CDOR"
filtered_data = filtered_data[((filtered_data['Curr'] == "USD") & (filtered_data['Leg 2'] == "USD-LIBOR-BBA") & (filtered_data["Days Between Trade and Effective"] == 2)
                               & (filtered_data["PF 1"] == "3M") & (filtered_data["PF 2"] == "6M")) |
                             ((filtered_data['Curr'] == "CAD") & (filtered_data['Leg 2'] == "CAD-BA-CDOR") & (filtered_data["Days Between Trade and Effective"] == 0) \
                              & (filtered_data["PF 1"] == "6M") & (filtered_data["PF 2"] == "6M"))]
filtered_data["T"] = filtered_data.apply(lambda x: int(x["T"][:-1]), axis=1)

filtered_data.drop(["File Date_ql", "Effective_ql", "Maturity_ql", "Trade Date_ql"], axis=1, inplace=True)

filtered_data.describe()


,Period,Rate 1,Rate 2,Group,Not.,Capped,SEF,Hour,ID,Othr Pmnt,T,Fair Rate,Difference,Days Between Trade and Effective
count,8586.000000,8586.000000,0.0,8586.000000,8.586000e+03,8586.000000,8586.0,8586.000000,8.586000e+03,0.0,8586.000000,8586.000000,8586.000000,8586.000000
mean,0.864547,1.784315,NaN,0.946774,6.819591e+07,0.393664,0.0,10.906010,3.426301e+06,NaN,6.618682,1.764432,1.988307,1.893548
std,0.342227,0.765933,NaN,0.224497,4.438932e+07,0.488590,0.0,3.482298,1.511520e+06,NaN,2.798942,0.737196,19.572606,0.448995
min,0.000000,0.170000,NaN,0.000000,1.000000e+04,0.000000,0.0,0.000000,7.532180e+05,NaN,1.000000,0.331830,-80.008691,0.000000
25%,1.000000,1.210000,NaN,1.000000,3.025000e+07,0.000000,0.0,9.000000,1.611904e+06,NaN,5.000000,1.214383,-0.614665,2.000000
50%,1.000000,1.800775,NaN,1.000000,6.700000e+07,0.000000,0.0,11.000000,3.318452e+06,NaN,5.000000,1.774453,1.049059,2.000000
75%,1.000000,2.364000,NaN,1.000000,1.000000e+08,1.000000,0.0,14.000000,5.199686e+06,NaN,10.000000,2.336436,2.875591,2.000000
max,1.000000,6.830000,NaN,1.000000,2.600000e+08,1.000000,0.0,21.000000,5.448567e+06,NaN,10.000000,3.307009,485.755577,2.000000


In [14]:
filtered_data.to_excel("data/filtered_data.xlsx")